In [ ]:
# =============================================================================
# Project: Trend-Context Fusion Network (TCFN) for Solar Power Forecasting
# File: main_tcfn_pipeline.ipynb
# Description: This script implements the full TCFN pipeline, including:
#              1. Data Preprocessing & Cyclical Encoding
#              2. Grid Search for Hyperparameter Optimization
#              3. Retraining Strategy on Combined Dataset
#              4. Final Evaluation (RMSE, MAE, R2)
#              5. XAI Analysis using SHAP (Feature Importance & Local Explanation)
# Environment: Google Colab / TensorFlow 2.x
# =============================================================================

# ---------------------------------------------------------
# 0. Setup & Configuration
# ---------------------------------------------------------
!pip install shap -q

import os
import random
import io
import numpy as np
import pandas as pd
import tensorflow as tf
import shap
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Dense, Dropout, LayerNormalization,
                                     MultiHeadAttention, Add, Concatenate,
                                     LSTM, Conv1D, BatchNormalization)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Robust Reproducibility Setup
SEED = 42
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    try:
        tf.config.experimental.enable_op_determinism()
    except:
        pass

set_seeds()
SEQ_LENGTH = 24

In [ ]:
# ---------------------------------------------------------
# 1. Data Loading & Feature Engineering
# ---------------------------------------------------------
print("\n--- [Step 1] Data Acquisition & Preprocessing ---")
print("Please upload the dataset (e.g., 'Dangjin_Landfill_PV_Dataset.csv').")

uploaded = files.upload()
filename = next(iter(uploaded))
print(f" >> File '{filename}' uploaded successfully.")

df = pd.read_csv(io.BytesIO(uploaded[filename]))

def apply_feature_engineering(df):
    """
    Applies cyclical time encoding to capture seasonal and diurnal patterns.
    """
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    df['DayOfYear'] = df['Date'].dt.dayofyear
    df['DaysInYear'] = df['Date'].dt.is_leap_year.apply(lambda x: 366 if x else 365)

    # Yearly Seasonality
    df['Day_sin'] = np.sin(2 * np.pi * df['DayOfYear'] / df['DaysInYear'])
    df['Day_cos'] = np.cos(2 * np.pi * df['DayOfYear'] / df['DaysInYear'])

    # Daily Seasonality
    df['Hour_sin'] = np.sin(2 * np.pi * df['Hour'] / 24.0)
    df['Hour_cos'] = np.cos(2 * np.pi * df['Hour'] / 24.0)
    return df

df = apply_feature_engineering(df)

# Define Features and Target
weather_cols = ['AverageTemp', 'LowTemp', 'HighTemp', 'RainFall', 'SteamPress',
                'DewPoint', 'Sunshine', 'Insolation', 'Cloudiness', 'GroundTemp',
                'Temp', 'Wind', 'Press', 'Humi']
time_cols = ['Day_sin', 'Day_cos', 'Hour_sin', 'Hour_cos']
feature_cols = weather_cols + time_cols
target_col = 'Solar_Power'

# Chronological Split
train_df = df[df['Year'].isin([2015, 2016, 2017])]
val_df = df[df['Year'] == 2018]
test_df = df[df['Year'] == 2019]

print(f" >> Data Split | Train: {train_df.shape}, Val: {val_df.shape}, Test: {test_df.shape}")

# Standardization
scaler = StandardScaler()
X_train_raw = scaler.fit_transform(train_df[feature_cols].values)
X_val_raw = scaler.transform(val_df[feature_cols].values)
X_test_raw = scaler.transform(test_df[feature_cols].values)

y_train_raw = train_df[target_col].values
y_val_raw = val_df[target_col].values
y_test_raw = test_df[target_col].values

def create_sequences(data, target, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        xs.append(data[i:(i + seq_length)])
        ys.append(target[i + seq_length])
    return np.array(xs), np.array(ys)

X_train, y_train = create_sequences(X_train_raw, y_train_raw, SEQ_LENGTH)
X_val, y_val = create_sequences(X_val_raw, y_val_raw, SEQ_LENGTH)
X_test, y_test = create_sequences(X_test_raw, y_test_raw, SEQ_LENGTH)

In [ ]:
# ---------------------------------------------------------
# 2. Model Architecture: TCFN
# ---------------------------------------------------------
def trend_aware_attention_block(x, num_heads=4, key_dim=16):
    """Adds a residual Multi-Head Attention block with LayerNorm."""
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(x, x)
    x = Add()([x, attn_output])
    x = LayerNormalization()(x)

    ffn = Dense(x.shape[-1], activation='swish')(x)
    x = Add()([x, ffn])
    x = LayerNormalization()(x)
    return x

def build_tcfn_model(input_shape, params):
    """Builds the Trend-Context Fusion Network based on hyperparameters."""
    inputs = Input(shape=input_shape)

    # 1. Local Trend Extraction (Conv1D)
    trend = Conv1D(params['lstm_units'], kernel_size=3, padding='same', activation='swish')(inputs)
    trend = BatchNormalization()(trend)
    trend = Dropout(params['dropout'])(trend)

    # 2. Global Context Mining (Attention)
    context = trend_aware_attention_block(inputs)

    # 3. Fusion & Sequential Modeling (LSTM)
    combined = Concatenate()([trend, context])
    x = LSTM(params['lstm_units'])(combined)
    x = Dropout(params['dropout'])(x)

    # 4. Regression Head
    x = Dense(params['lstm_units'], activation='swish')(x)
    outputs = Dense(1, activation='relu')(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(params['lr']), loss='mse', metrics=['mae'])
    return model

In [ ]:
# ---------------------------------------------------------
# 3. Hyperparameter Grid Search
# ---------------------------------------------------------
print("\n--- [Step 2] Hyperparameter Grid Search (Experimental Design) ---")

scenarios = [
    {'id': 'Light_Fast',        'lstm_units': 32,  'lr': 0.001,  'dropout': 0.2, 'desc': 'Fast Convergence'},
    {'id': 'Standard_Balanced', 'lstm_units': 64,  'lr': 0.0005, 'dropout': 0.3, 'desc': 'Baseline Capacity'},
    {'id': 'Deep_Slow',         'lstm_units': 128, 'lr': 0.0001, 'dropout': 0.3, 'desc': 'Fine-tuning'},
    {'id': 'Deep_Robust',       'lstm_units': 128, 'lr': 0.0005, 'dropout': 0.5, 'desc': 'High Regularization'}
]

best_val_loss = float('inf')
best_params = {}
best_epoch_num = 0

for sc in scenarios:
    print(f"\n>> Experiment: {sc['id']} ({sc['desc']})")

    model = build_tcfn_model((SEQ_LENGTH, len(feature_cols)), sc)
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    hist = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=64,
        callbacks=[early_stop],
        verbose=0
    )

    min_loss = min(hist.history['val_loss'])
    opt_epoch = np.argmin(hist.history['val_loss']) + 1

    print(f"   -> Val MSE: {min_loss:.4f} (at Epoch {opt_epoch})")

    if min_loss < best_val_loss:
        best_val_loss = min_loss
        best_params = sc
        best_epoch_num = opt_epoch

print(f"\n[Selected Configuration] {best_params['id']}")
print(f" >> Optimal Retraining Epochs: {best_epoch_num}")

In [ ]:
# ---------------------------------------------------------
# 4. Retraining Strategy (Train + Val)
# ---------------------------------------------------------
print("\n--- [Step 3] Retraining on Combined Dataset ---")

X_combined = np.concatenate((X_train, X_val), axis=0)
y_combined = np.concatenate((y_train, y_val), axis=0)

final_model = build_tcfn_model((SEQ_LENGTH, len(feature_cols)), best_params)

final_model.fit(
    X_combined, y_combined,
    epochs=best_epoch_num,
    batch_size=32,
    verbose=1
)

In [ ]:
# ---------------------------------------------------------
# 5. Final Evaluation
# ---------------------------------------------------------
print("\n--- [Step 4] Final Evaluation on Test Set (2019) ---")

y_pred = final_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f" >> RMSE: {rmse:.4f}")
print(f" >> MAE : {mae:.4f}")
print(f" >> R2  : {r2:.4f}")

# Visualization
plt.figure(figsize=(14, 5))
plt.plot(y_test[:200], label='Actual Power', color='navy', alpha=0.7)
plt.plot(y_pred[:200], label='TCFN Prediction', color='crimson', alpha=0.7, linestyle='--')
plt.title(f'Forecast Comparison (First 200 Hours) | R2: {r2:.3f}', fontsize=12, fontweight='bold')
plt.ylabel('Solar Power (kW)')
plt.xlabel('Time (Hours)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# ---------------------------------------------------------
# 6. Explainable AI (SHAP)
# ---------------------------------------------------------
print("\n--- [Step 5] XAI Analysis: SHAP ---")
print("Computing SHAP values (this may take time)...")

# Wrapper for SHAP (3D -> 2D -> 3D)
def model_predict_wrapper(data_flattened):
    data_reshaped = data_flattened.reshape(-1, SEQ_LENGTH, len(feature_cols))
    return final_model.predict(data_reshaped, verbose=0).flatten()

# Background & Test Samples
background_data = X_train[np.random.choice(X_train.shape[0], 50, replace=False)]
background_flat = background_data.reshape(50, -1)

test_idx = np.random.choice(X_test.shape[0], 10, replace=False)
test_samples = X_test[test_idx]
test_flat = test_samples.reshape(10, -1)

explainer = shap.KernelExplainer(model_predict_wrapper, background_flat)
shap_values = explainer.shap_values(test_flat, nsamples=100)

# --- Visualization 1: Global Feature Importance ---
print("\n[Visualizing Global Feature Importance]")
shap_matrix = np.array(shap_values)
shap_3d = shap_matrix.reshape(-1, SEQ_LENGTH, len(feature_cols))
feature_imp = np.abs(shap_3d).mean(axis=(0, 1))

indices = np.argsort(feature_imp)
sorted_feats = [feature_cols[i] for i in indices]
sorted_vals = feature_imp[indices]

plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_cols)), sorted_vals, color='steelblue', align='center')
plt.yticks(range(len(feature_cols)), sorted_feats)
plt.xlabel('Mean |SHAP Value| (Impact on Output)')
plt.title('Global Feature Importance', fontsize=12, fontweight='bold')
plt.show()

# --- Visualization 2: Local Explanation (Waterfall) ---
print("\n[Visualizing Local Explanation (Sample #0)]")
sample_id = 0
feat_names_flat = [f"{col}_t{t}" for t in range(SEQ_LENGTH) for col in feature_cols]

explanation = shap.Explanation(
    values=shap_values[sample_id],
    base_values=explainer.expected_value,
    data=test_flat[sample_id],
    feature_names=feat_names_flat
)

plt.figure(figsize=(8, 8))
shap.plots.waterfall(explanation, max_display=15, show=False)
plt.title(f'Local Explanation for Sample Index {test_idx[sample_id]}', fontsize=12, fontweight='bold')
plt.show()

print("\nAll analysis completed successfully.")